In [1]:
import pyspark
import pandas as pd
import numpy as np
from pydataset import data

In [2]:
import pyspark

spark = pyspark.sql.SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/05 08:58:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


1) Create a spark data frame that contains your favorite programming languages.

* The name of the column should be language

In [4]:
pdf = pd.DataFrame({'language': ['Python', 'Java', 'SQL', 'R', 'Ruby', 'PowerShell', 'PySpark', 'JavaScript', 'C++']})

In [5]:
df = spark.createDataFrame(pdf)

* View the schema of the dataframe

In [6]:
df.show()

+----------+
|  language|
+----------+
|    Python|
|      Java|
|       SQL|
|         R|
|      Ruby|
|PowerShell|
|   PySpark|
|JavaScript|
|       C++|
+----------+



* Output the shape of the dataframe

In [7]:
df.count(), len(df.columns)

(9, 1)

* Show the first five records in the dataframe

In [8]:
df.show(5)

+--------+
|language|
+--------+
|  Python|
|    Java|
|     SQL|
|       R|
|    Ruby|
+--------+
only showing top 5 rows



2) Load the `mpg` dataset as a spark dataframe.

a. Create 1 column of output that contains a message like the one below:

`The 1999 audi a4 has a 4 cylinder engine.`

For each vehicle.

In [9]:
from pydataset import data

mpg = spark.createDataFrame(data("mpg"))

In [10]:
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [11]:
from pyspark.sql.functions import col, expr, min, max, sum, count, mean, avg, concat, lit

In [12]:
mpg.select(
    concat(lit("This "), 
           mpg.year, 
           lit(" "), 
           mpg.manufacturer, 
           lit(" has a "), 
           mpg.cyl, 
           lit(" cylinder engine.")).alias('message')).show(truncate=False)

+--------------------------------------------+
|message                                     |
+--------------------------------------------+
|This 1999 audi has a 4 cylinder engine.     |
|This 1999 audi has a 4 cylinder engine.     |
|This 2008 audi has a 4 cylinder engine.     |
|This 2008 audi has a 4 cylinder engine.     |
|This 1999 audi has a 6 cylinder engine.     |
|This 1999 audi has a 6 cylinder engine.     |
|This 2008 audi has a 6 cylinder engine.     |
|This 1999 audi has a 4 cylinder engine.     |
|This 1999 audi has a 4 cylinder engine.     |
|This 2008 audi has a 4 cylinder engine.     |
|This 2008 audi has a 4 cylinder engine.     |
|This 1999 audi has a 6 cylinder engine.     |
|This 1999 audi has a 6 cylinder engine.     |
|This 2008 audi has a 6 cylinder engine.     |
|This 2008 audi has a 6 cylinder engine.     |
|This 1999 audi has a 6 cylinder engine.     |
|This 2008 audi has a 6 cylinder engine.     |
|This 2008 audi has a 8 cylinder engine.     |
|This 2008 ch

b. Transform the trans column so that it only contains either manual or auto.

In [13]:
from pyspark.sql.functions import regexp_extract, regexp_replace

In [14]:
mpg.select(mpg.trans,
          regexp_extract('trans', r'^(.*)\(', 1).alias('transmission')).show()

+----------+------------+
|     trans|transmission|
+----------+------------+
|  auto(l5)|        auto|
|manual(m5)|      manual|
|manual(m6)|      manual|
|  auto(av)|        auto|
|  auto(l5)|        auto|
|manual(m5)|      manual|
|  auto(av)|        auto|
|manual(m5)|      manual|
|  auto(l5)|        auto|
|manual(m6)|      manual|
|  auto(s6)|        auto|
|  auto(l5)|        auto|
|manual(m5)|      manual|
|  auto(s6)|        auto|
|manual(m6)|      manual|
|  auto(l5)|        auto|
|  auto(s6)|        auto|
|  auto(s6)|        auto|
|  auto(l4)|        auto|
|  auto(l4)|        auto|
+----------+------------+
only showing top 20 rows



3) Load the `tips` dataset as a spark dataframe.

a. What percentage of observations are smokers?

In [15]:
tips = spark.createDataFrame(data('tips'))

tips.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



In [16]:
round(tips.filter(tips.smoker == 'Yes').count()/tips.count()*100, 2)

38.11

b. Create a column that contains the tip percentage

In [17]:
tips.select((tips.tip/tips.total_bill*100).alias("tip_percentage")).show()

+------------------+
|    tip_percentage|
+------------------+
|5.9446733372572105|
|16.054158607350097|
|16.658733936220845|
| 13.97804054054054|
|14.680764538430255|
| 18.62396204033215|
| 22.80501710376283|
|11.607142857142858|
|13.031914893617023|
|21.853856562922868|
| 16.65043816942551|
|14.180374361883155|
|10.181582360570687|
|16.277807921866522|
|20.364126770060686|
|18.164967562557923|
| 16.16650532429816|
|22.774708410067525|
|20.624631703005306|
|16.222760290556902|
+------------------+
only showing top 20 rows



c. Calculate the average tip percentage for each combination of sex and smoker.

In [18]:
tips.groupBy('sex', 'smoker').mean('tip').show()

+------+------+-----------------+
|   sex|smoker|         avg(tip)|
+------+------+-----------------+
|  Male|    No| 3.11340206185567|
|Female|    No|2.773518518518518|
|  Male|   Yes|3.051166666666667|
|Female|   Yes|2.931515151515151|
+------+------+-----------------+



4) Use the seattle weather dataset referenced in the lesson to answer the questions below.

* Convert the temperatures to fahrenheit.

In [19]:
from vega_datasets import data

In [20]:
weather = spark.createDataFrame(data.seattle_weather())

In [21]:
weather.show(5)

+-------------------+-------------+--------+--------+----+-------+
|               date|precipitation|temp_max|temp_min|wind|weather|
+-------------------+-------------+--------+--------+----+-------+
|2012-01-01 00:00:00|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02 00:00:00|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03 00:00:00|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04 00:00:00|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05 00:00:00|          1.3|     8.9|     2.8| 6.1|   rain|
+-------------------+-------------+--------+--------+----+-------+
only showing top 5 rows



In [22]:
weather.select(
    ((weather.temp_max * (9/5)) + 32),
    ((weather.temp_min * (9/5)) + 32)
).show()

+-----------------------+-----------------------+
|((temp_max * 1.8) + 32)|((temp_min * 1.8) + 32)|
+-----------------------+-----------------------+
|     55.040000000000006|                   41.0|
|                  51.08|                  37.04|
|                  53.06|                  44.96|
|                  53.96|                  42.08|
|     48.019999999999996|                  37.04|
|                  39.92|                  35.96|
|                  44.96|                  37.04|
|                   50.0|                  37.04|
|                  48.92|                   41.0|
|     42.980000000000004|                  33.08|
|     42.980000000000004|                  30.02|
|     42.980000000000004|                  28.94|
|                   41.0|                  26.96|
|                  39.92|                  33.08|
|                  33.98|     26.060000000000002|
|                  35.06|                  26.96|
|                  37.94|                   32.0|


* Which month has the most rain, on average?

In [23]:
weather.select(weather.date).show()

+-------------------+
|               date|
+-------------------+
|2012-01-01 00:00:00|
|2012-01-02 00:00:00|
|2012-01-03 00:00:00|
|2012-01-04 00:00:00|
|2012-01-05 00:00:00|
|2012-01-06 00:00:00|
|2012-01-07 00:00:00|
|2012-01-08 00:00:00|
|2012-01-09 00:00:00|
|2012-01-10 00:00:00|
|2012-01-11 00:00:00|
|2012-01-12 00:00:00|
|2012-01-13 00:00:00|
|2012-01-14 00:00:00|
|2012-01-15 00:00:00|
|2012-01-16 00:00:00|
|2012-01-17 00:00:00|
|2012-01-18 00:00:00|
|2012-01-19 00:00:00|
|2012-01-20 00:00:00|
+-------------------+
only showing top 20 rows



In [24]:
from pyspark.sql.functions import month, year, quarter

In [77]:
weather.withColumn("month", month('date'))\
.groupBy('month')\
.agg(mean('precipitation').alias('average_rain'))\ #aliasing can only be done with aggregate function when done this way
.sort('average_rain', ascending=False)\ #could also do sort(col('average_rain').desc())
.show()

+-----+-------------------+
|month|       average_rain|
+-----+-------------------+
|   11|  5.354166666666667|
|   12|  5.021774193548389|
|    3|  4.888709677419355|
|   10|  4.059677419354839|
|    1| 3.7580645161290316|
|    2|  3.734513274336283|
|    4|  3.128333333333333|
|    9| 1.9624999999999997|
|    5| 1.6733870967741935|
|    8| 1.3201612903225806|
|    6| 1.1075000000000002|
|    7|0.38870967741935486|
+-----+-------------------+



* Which year was the windiest?

In [26]:
weather.withColumn("year", year('date'))\
.groupBy('year')\
.sum('wind')\
.sort(sum('wind').desc())\
.show()

+----+------------------+
|year|         sum(wind)|
+----+------------------+
|2012|            1244.7|
|2014|1236.5000000000007|
|2015|1153.3000000000002|
|2013|1100.8000000000006|
+----+------------------+



* What is the most frequent type of weather in January?

In [105]:
weather.filter(month(weather.date) == 1)\
.groupBy(weather.weather).count().sort('count', ascending=False).show()

+-------+-----+
|weather|count|
+-------+-----+
|    fog|   38|
|   rain|   35|
|    sun|   33|
|drizzle|   10|
|   snow|    8|
+-------+-----+



* What is the average high and low temperature on sunny days in July in 2013 and 2014?

In [28]:
weather.filter((month(weather.date) == 7) & (year(weather.date) == (2013 | 2014)) & (weather.weather == 'sun'))\
.groupBy(weather.weather).mean('temp_max', 'temp_min').show()

+-------+-------------+------------------+
|weather|avg(temp_max)|     avg(temp_min)|
+-------+-------------+------------------+
|    sun|       28.884|15.867999999999995|
+-------+-------------+------------------+



* What percentage of days were rainy in q3 of 2015?

In [84]:
weather.filter((quarter('date') == 3) & (year('date') == 2015) & (weather.weather == ('rain' or 'drizzle')))\
.count()/weather.filter((quarter('date') == 3) & (year('date') == 2015)).count()*100

2.1739130434782608

* For each year, find what percentage of days it rained (had non-zero precipitation).

In [101]:
weather.filter(weather.precipitation > 0).groupBy(year('date')).count().show()

+----------+-----+
|year(date)|count|
+----------+-----+
|      2012|  177|
|      2013|  152|
|      2014|  150|
|      2015|  144|
+----------+-----+



In [103]:
weather.groupBy(year('date')).count().show()

+----------+-----+
|year(date)|count|
+----------+-----+
|      2012|  366|
|      2013|  365|
|      2014|  365|
|      2015|  365|
+----------+-----+



In [104]:
(weather.filter(weather.precipitation > 0).groupBy(year('date')).count()) / (weather.groupBy(year('date')).count())

TypeError: unsupported operand type(s) for /: 'DataFrame' and 'DataFrame'

In [ ]:
# What percentage of days were rainy in Q3 of 2015?
total_days_q3_2015 = weather.filter((year("date") == 2015) & (month("date").between(7, 9))).count()
rainy_days_q3_2015 = weather.filter((year("date") == 2015) & (month("date").between(7, 9)) & (weather.precipitation > 0)).count()
rainy_percentage_q3_2015 = (rainy_days_q3_2015 / total_days_q3_2015) * 100
print("Percentage of rainy days in Q3 of 2015:", rainy_percentage_q3_2015)
# For each year, find the percentage of days it rained (had non-zero precipitation)
rainy_days_percentage_by_year = weather.groupBy(year("date").alias("year")).agg((sum(when(weather.precipitation > 0, 1)) / count("*") * 100).alias("rainy_percentage"))
rainy_days_percentage_by_year.show()